In [1]:
##newest 06/03 !!!!final

import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
# import pyodbc
# import random
# import zipfile
# import io
import re
# import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/HOSPITALS/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)

ICDPL_dfs = []

# Define the regex patterns for ICD9 and ICD10 codes using non-capturing groups
icd9_pattern = re.compile(r'(?:^|,)\s*(3320|332\.0)\s*(?:,|$)', re.IGNORECASE)
icd10_pattern = re.compile(r'(?:^|,)\s*G20(?:\s*|,|$)', re.IGNORECASE)

# Loop through the partitions and process the data
for i in tqdm(range(1, len(all_keys_ICD) + 1)):
    ICDPL_df = reader_ICD[f'ICD_partition_{i}']
    
    # Strip leading/trailing spaces and convert to string
    ICDPL_df['ICDCD'] = ICDPL_df['ICDCD'].astype(str).str.strip()
    
    # Filter with regex pattern for combined ICD9 and ICD10 codes
    icd9_matches = ICDPL_df['ICDCD'].str.contains(icd9_pattern)
    icd10_matches = ICDPL_df['ICDCD'].str.contains(icd10_pattern)
    
    # Include rows that match either ICD9 or ICD10 patterns
    included_df = ICDPL_df[icd9_matches | icd10_matches]
    
    ICDPL_dfs.append(included_df)

# Concatenate all DataFrames
ICDPL_df = pd.concat(ICDPL_dfs, axis=0, ignore_index=True)

# Print the unique 'ICDCD' values and the DataFrame
print(ICDPL_df['ICDCD'].unique())
print(ICDPL_df)

['ICD_partition_1', 'ICD_partition_10', 'ICD_partition_100', 'ICD_partition_101', 'ICD_partition_102', 'ICD_partition_103', 'ICD_partition_104', 'ICD_partition_105', 'ICD_partition_106', 'ICD_partition_107', 'ICD_partition_108', 'ICD_partition_109', 'ICD_partition_11', 'ICD_partition_110', 'ICD_partition_111', 'ICD_partition_112', 'ICD_partition_113', 'ICD_partition_114', 'ICD_partition_115', 'ICD_partition_116', 'ICD_partition_117', 'ICD_partition_118', 'ICD_partition_119', 'ICD_partition_12', 'ICD_partition_120', 'ICD_partition_121', 'ICD_partition_122', 'ICD_partition_123', 'ICD_partition_124', 'ICD_partition_125', 'ICD_partition_126', 'ICD_partition_127', 'ICD_partition_128', 'ICD_partition_129', 'ICD_partition_13', 'ICD_partition_130', 'ICD_partition_131', 'ICD_partition_132', 'ICD_partition_133', 'ICD_partition_134', 'ICD_partition_135', 'ICD_partition_136', 'ICD_partition_137', 'ICD_partition_138', 'ICD_partition_139', 'ICD_partition_14', 'ICD_partition_140', 'ICD_partition_141'

  0%|          | 0/511 [00:00<?, ?it/s]/tmp/ipykernel_1835/936766447.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  icd9_matches = ICDPL_df['ICDCD'].str.contains(icd9_pattern)
  0%|          | 1/511 [00:01<13:54,  1.64s/it]/tmp/ipykernel_1835/936766447.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  icd9_matches = ICDPL_df['ICDCD'].str.contains(icd9_pattern)
  0%|          | 2/511 [00:03<14:34,  1.72s/it]/tmp/ipykernel_1835/936766447.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  icd9_matches = ICDPL_df['ICDCD'].str.contains(icd9_pattern)
  1%|          | 3/511 [00:05<14:20,  1.69s/it]/tmp/ipykernel_1835/936766447.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups.

['332.0' 'G20.C' 'G20.A1' 'G20.B1' 'G20.B2' 'G20.A2' 'G20']
        BDSPEncounterID  EncounterLineNBR  BDSPPatientID  \
0          1.339649e+10                 1    113319054.0   
1          1.347001e+10                 1    115367785.0   
2          1.338726e+10                 1    117147113.0   
3          1.366315e+10                 3    118192867.0   
4          1.354173e+10                 1    112059081.0   
...                 ...               ...            ...   
442131     1.354375e+10                 1    121067212.0   
442132     1.331023e+10                 1    112506882.0   
442133     1.324917e+10                 9    119343967.0   
442134     1.363821e+10                 1    111983910.0   
442135     1.336804e+10                 1    119979436.0   

                  ShiftedContactDTS  ICDLineNBR   ICDCD             ICDDSC  \
0       2017-09-21 00:00:00.0000000         1.0   332.0  Paralysis agitans   
1       2020-02-18 00:00:00.0000000         1.0   332.0  Paraly

In [2]:
#define timewindow
start_date = pd.Timestamp('2016-05-01')
end_date = pd.Timestamp('2023-01-01')

ICDPL_df['ShiftedContactDTS'] = pd.to_datetime(ICDPL_df['ShiftedContactDTS'])

# FILTER BASED ON TIME WINDOW
filt_ICDPL_df = ICDPL_df[(ICDPL_df['ShiftedContactDTS'] >= start_date) & (ICDPL_df['ShiftedContactDTS'] < end_date)]

print(filt_ICDPL_df['ICDCD'])
print(filt_ICDPL_df)

0          332.0
1          332.0
2          332.0
4          332.0
5          332.0
           ...  
442131     G20.C
442132    G20.A1
442133    G20.A1
442134    G20.A1
442135    G20.A1
Name: ICDCD, Length: 365192, dtype: object
        BDSPEncounterID  EncounterLineNBR  BDSPPatientID ShiftedContactDTS  \
0          1.339649e+10                 1    113319054.0        2017-09-21   
1          1.347001e+10                 1    115367785.0        2020-02-18   
2          1.338726e+10                 1    117147113.0        2017-09-14   
4          1.354173e+10                 1    112059081.0        2022-05-31   
5          1.339986e+10                 4    119027616.0        2019-09-29   
...                 ...               ...            ...               ...   
442131     1.354375e+10                 1    121067212.0        2021-04-15   
442132     1.331023e+10                 1    112506882.0        2017-07-06   
442133     1.324917e+10                 9    119343967.0        2017

In [3]:
# GETCSVOFALLPOSDATA

filt_ICDPL_df = filt_ICDPL_df.rename(columns={'ShiftedContactDTS': 'CreateDate'})

filt_ICDPL_df.to_csv('MGB_ICDPLUS.csv', index=False)

In [13]:
unique_icdcd = filt_ICDPL_df['ICDCD'].unique()
print(list(unique_icdcd))
print(filt_ICDPL_df)

['332.0', 'G20.C', 'G20.A1', 'G20.B1', 'G20.A2', 'G20.B2', 'G20']
        BDSPEncounterID  EncounterLineNBR  BDSPPatientID ShiftedContactDTS  \
0          1.339649e+10                 1    113319054.0        2017-09-21   
1          1.347001e+10                 1    115367785.0        2020-02-18   
2          1.338726e+10                 1    117147113.0        2017-09-14   
4          1.354173e+10                 1    112059081.0        2022-05-31   
5          1.339986e+10                 4    119027616.0        2019-09-29   
...                 ...               ...            ...               ...   
442131     1.354375e+10                 1    121067212.0        2021-04-15   
442132     1.331023e+10                 1    112506882.0        2017-07-06   
442133     1.324917e+10                 9    119343967.0        2017-10-28   
442134     1.363821e+10                 1    111983910.0        2022-09-07   
442135     1.336804e+10                 1    119979436.0        2018-05-13  

In [9]:
unique_ids_ICD_plus = set(filt_ICDPL_df['BDSPPatientID'].unique())

In [10]:
print(len(unique_ids_ICD_plus))
ICDPL_MGB_LIST= list(unique_ids_ICD_plus)

ICDPL_MGB_df = pd.DataFrame(ICDPL_MGB_LIST, columns=['BDSPPatientID'])
print(len(ICDPL_MGB_df))
ICDPL_MGB_df.to_csv('ICDPL_MGB_New.csv', index=False)

14627
14627


In [4]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pyodbc
import random
import zipfile
import io
import re
import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)


## icdM

ICDMN_dfs = []

pattern = re.compile(r'^(332\.?0|G20|G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2)$', re.IGNORECASE)  # pattern = re.compile(r'^(332\.?0|G20)$', re.IGNORECASE)
# G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2
for i in tqdm(range (1, len(all_keys_ICD) + 1)):
    ICDMN_df = reader_ICD[f'ICD_partition_{i}']
    
    # Debugging: Print unique DiagnosisCode values
    # print(f"Unique DiagnosisCode values in partition {i}:")
    # print(ICDPL_df['DiagnosisCode'].unique())
    
    # Strip leading/trailing spaces and convert to string
    ICDMN_df['ICDCD'] = ICDMN_df['ICDCD'].astype(str).str.strip()
    
    # Filter with regex pattern
    ICDMN_df = ICDMN_df[~ICDMN_df.ICDCD.str.match(pattern)]
    
    ICDMN_dfs.append(ICDMN_df)

ICDMN_df = pd.concat(ICDMN_dfs, axis=0, ignore_index=True)

# # Print the 'DiagnosisCode' column
print(ICDMN_df['ICDCD'].unique())
print(ICDMN_df)

['ICD_partition_1', 'ICD_partition_10', 'ICD_partition_100', 'ICD_partition_101', 'ICD_partition_102', 'ICD_partition_103', 'ICD_partition_104', 'ICD_partition_105', 'ICD_partition_106', 'ICD_partition_107', 'ICD_partition_108', 'ICD_partition_109', 'ICD_partition_11', 'ICD_partition_110', 'ICD_partition_111', 'ICD_partition_112', 'ICD_partition_113', 'ICD_partition_114', 'ICD_partition_115', 'ICD_partition_116', 'ICD_partition_117', 'ICD_partition_118', 'ICD_partition_119', 'ICD_partition_12', 'ICD_partition_120', 'ICD_partition_121', 'ICD_partition_122', 'ICD_partition_123', 'ICD_partition_124', 'ICD_partition_125', 'ICD_partition_126', 'ICD_partition_127', 'ICD_partition_128', 'ICD_partition_129', 'ICD_partition_13', 'ICD_partition_130', 'ICD_partition_131', 'ICD_partition_132', 'ICD_partition_133', 'ICD_partition_134', 'ICD_partition_135', 'ICD_partition_136', 'ICD_partition_137', 'ICD_partition_138', 'ICD_partition_139', 'ICD_partition_14', 'ICD_partition_140', 'ICD_partition_141'

 42%|████▏     | 217/511 [07:50<13:19,  2.72s/it]

: 

In [1]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pyodbc
import random
import zipfile
import io
import re
import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)


## icdmn

ICDMN_dfs = []

pattern = re.compile(r'^(332\.?0|G20|G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2)$', re.IGNORECASE)

chunk_size = 50  # Adjust chunk size based on your system's memory capacity

# Loop through each partition in chunks and process DataFrames
for i in tqdm(range(0, len(all_keys_ICD), chunk_size), desc="Processing ICD partitions"):
    chunk_keys = all_keys_ICD[i:i + chunk_size]
    
    chunk_dfs = []
    for key in chunk_keys:
        ICDMN_df = reader_ICD[key]
        
        # Strip leading/trailing spaces and convert to string
        ICDMN_df['ICDCD'] = ICDMN_df['ICDCD'].astype(str).str.strip()
        
        # Filter with regex pattern
        ICDMN_df = ICDMN_df[~ICDMN_df.ICDCD.str.match(pattern)]
        
        chunk_dfs.append(ICDMN_df)
    
    # Concatenate the chunk DataFrames into a single DataFrame
    chunk_df = pd.concat(chunk_dfs, axis=0, ignore_index=True)
    
    # Append the concatenated chunk DataFrame to the list
    ICDMN_dfs.append(chunk_df)
    
    # Clear chunk_dfs to free up memory
    del chunk_dfs

# Concatenate all processed DataFrames into a single DataFrame
ICDMN_df = pd.concat(ICDMN_dfs, axis=0, ignore_index=True)

# Print the unique 'ICDCD' values
print(ICDMN_df['ICDCD'].unique())
print(ICDMN_df)

['ICD_partition_1', 'ICD_partition_10', 'ICD_partition_100', 'ICD_partition_101', 'ICD_partition_102', 'ICD_partition_103', 'ICD_partition_104', 'ICD_partition_105', 'ICD_partition_106', 'ICD_partition_107', 'ICD_partition_108', 'ICD_partition_109', 'ICD_partition_11', 'ICD_partition_110', 'ICD_partition_111', 'ICD_partition_112', 'ICD_partition_113', 'ICD_partition_114', 'ICD_partition_115', 'ICD_partition_116', 'ICD_partition_117', 'ICD_partition_118', 'ICD_partition_119', 'ICD_partition_12', 'ICD_partition_120', 'ICD_partition_121', 'ICD_partition_122', 'ICD_partition_123', 'ICD_partition_124', 'ICD_partition_125', 'ICD_partition_126', 'ICD_partition_127', 'ICD_partition_128', 'ICD_partition_129', 'ICD_partition_13', 'ICD_partition_130', 'ICD_partition_131', 'ICD_partition_132', 'ICD_partition_133', 'ICD_partition_134', 'ICD_partition_135', 'ICD_partition_136', 'ICD_partition_137', 'ICD_partition_138', 'ICD_partition_139', 'ICD_partition_14', 'ICD_partition_140', 'ICD_partition_141'

Processing ICD partitions: 100%|██████████| 11/11 [17:54<00:00, 97.69s/it] 


In [1]:
print(ICDMN_df)

NameError: name 'ICDMN_df' is not defined

In [ ]:
ICDMN_df['ShiftedContactDTS'] = pd.to_datetime(ICDMN_df['ShiftedContactDTS'])

# FILTER BASED ON TIME WINDOW
filt_ICDMN_df = ICDMN_df[(ICDMN_df['ShiftedContactDTS'] >= start_date) & (ICDMN_df['ShiftedContactDTS'] < end_date)]

print(filt_ICDMN_df['ICDCD'])
print(filt_ICDMN_df)

In [2]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pandas as pd
import pyodbc
import random
import zipfile
import io
import re 
import csv

In [3]:
reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)

['ICD_partition_1', 'ICD_partition_10', 'ICD_partition_100', 'ICD_partition_101', 'ICD_partition_102', 'ICD_partition_103', 'ICD_partition_104', 'ICD_partition_105', 'ICD_partition_106', 'ICD_partition_107', 'ICD_partition_108', 'ICD_partition_109', 'ICD_partition_11', 'ICD_partition_110', 'ICD_partition_111', 'ICD_partition_112', 'ICD_partition_113', 'ICD_partition_114', 'ICD_partition_115', 'ICD_partition_116', 'ICD_partition_117', 'ICD_partition_118', 'ICD_partition_119', 'ICD_partition_12', 'ICD_partition_120', 'ICD_partition_121', 'ICD_partition_122', 'ICD_partition_123', 'ICD_partition_124', 'ICD_partition_125', 'ICD_partition_126', 'ICD_partition_127', 'ICD_partition_128', 'ICD_partition_129', 'ICD_partition_13', 'ICD_partition_130', 'ICD_partition_131', 'ICD_partition_132', 'ICD_partition_133', 'ICD_partition_134', 'ICD_partition_135', 'ICD_partition_136', 'ICD_partition_137', 'ICD_partition_138', 'ICD_partition_139', 'ICD_partition_14', 'ICD_partition_140', 'ICD_partition_141'

In [4]:
### count total number of files

# Initialize the list to collect DataFrames
ICD_dfs_initial = []

# Loop through each partition and collect DataFrames
for i in range(1, len(all_keys_ICD) + 1):
    ICD_df = reader_ICD[f'ICD_partition_{i}']
    
    # Append the DataFrame to the list
    ICD_dfs_initial.append(ICD_df)

# Concatenate all initial DataFrames into a single DataFrame
ICD_df_initial = pd.concat(ICD_dfs_initial, axis=0, ignore_index=True)

# Count the total number of rows in the initial data
total_rows_initial = ICD_df_initial.shape[0]

# Print the total number of rows
print(f"Total number of rows in the initial data: {total_rows_initial}")
print(ICD_df_initial.columns())

print(ICD_df_initial.head())



: 

In [ ]:
# unique_combinations_icd10 = ICD_df_initial['icd10_code'].str.contains('332\.?0?0', case=False, na=False)
# unique_combinations_icd10 = ICD_df_initial[unique_combinations_icd10]['icd10_code'].unique()

# unique_combinations_icd9 = ICD_df_initial['icd9_code'].str.contains('G20', case=False, na=False)
# unique_combinations_icd9 = ICD_df_initial[unique_combinations_icd9]['icd9_code'].unique()

# # Print the unique combinations
# print(unique_combinations_icd10)
# print(unique_combinations_icd9)